In [2]:
from nst_scraper import nst_on_ice_scraper


In [3]:
player_stats_2023_df = nst_on_ice_scraper(fromseason=20232024, thruseason=20232024, startdate='', enddate='')


In [4]:
player_stats_2023_df

,player,team,position,gp,toi,goals,total_assists,first_assists,second_assists,total_points,...,misconduct,penalties_drawn,giveaways,takeaways,hits,hits_taken,shots_blocked,faceoffs_won,faceoffs_lost,faceoffs_%
0,Ryan Suter,DAL,D,82,1403.833333,2,14,6,8,16,...,0,0,28,14,75,92,86,0,0,-
1,Jeff Carter,PIT,C,72,664.900000,5,3,1,2,8,...,0,3,16,16,75,32,25,169,87,66.02
2,Zach Parise,COL,L,30,340.616667,4,3,1,2,7,...,0,2,4,10,20,22,23,11,21,34.38
3,Brent Burns,CAR,D,82,1276.683333,5,12,5,7,17,...,0,5,43,30,34,33,72,0,0,-
4,Corey Perry,"CHI, EDM",R,54,600.266667,9,7,4,3,16,...,0,14,17,15,42,66,17,0,1,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,Jiri Smejkal,OTT,L,20,164.316667,1,1,1,0,2,...,0,2,5,6,32,14,13,9,15,37.50
920,Nikolas Matinpalo,OTT,D,4,27.333333,0,0,0,0,0,...,0,0,1,0,4,7,2,0,0,-
921,Waltteri Merela,T.B,C,19,170.500000,1,0,0,0,1,...,0,4,2,1,30,32,6,1,3,25.00
922,Patrik Koch,ARI,D,1,9.333333,0,0,0,0,0,...,1,0,0,0,4,2,1,0,0,-


In [6]:
def filter_players_by_toi(df, min_toi=400):
    """
    Filters the input DataFrame to include only players with a total time on ice (toi) of at least min_toi.

    Parameters:
        df (pd.DataFrame): The DataFrame containing player statistics.
        min_toi (int, optional): The minimum total time on ice required to include a player. Defaults to 400.

    Returns:
        pd.DataFrame: A filtered DataFrame with players having toi >= min_toi.
    """
    filtered_df = df[df['toi'] >= min_toi]
    return filtered_df


In [7]:
player_stats_2023_df = filter_players_by_toi(player_stats_2023_df, min_toi=400)
player_stats_2023_df

,player,team,position,gp,toi,goals,total_assists,first_assists,second_assists,total_points,...,misconduct,penalties_drawn,giveaways,takeaways,hits,hits_taken,shots_blocked,faceoffs_won,faceoffs_lost,faceoffs_%
0,Ryan Suter,DAL,D,82,1403.833333,2,14,6,8,16,...,0,0,28,14,75,92,86,0,0,-
1,Jeff Carter,PIT,C,72,664.900000,5,3,1,2,8,...,0,3,16,16,75,32,25,169,87,66.02
3,Brent Burns,CAR,D,82,1276.683333,5,12,5,7,17,...,0,5,43,30,34,33,72,0,0,-
4,Corey Perry,"CHI, EDM",R,54,600.266667,9,7,4,3,16,...,0,14,17,15,42,66,17,0,1,0.00
5,Joe Pavelski,DAL,C,82,1085.316667,12,25,14,11,37,...,0,19,51,19,70,45,61,299,281,51.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,Andrei Kuzmenko,"CGY, VAN",L,72,815.650000,14,13,9,4,27,...,0,15,21,24,12,68,16,1,0,100.00
907,Connor Bedard,CHI,C,68,1043.150000,16,18,12,6,34,...,0,20,46,37,54,68,25,190,300,38.78
908,Zach Benson,BUF,L,71,886.483333,10,16,10,6,26,...,0,13,18,30,35,90,30,4,10,28.57
910,Leo Carlsson,ANA,C,55,788.383333,7,8,4,4,15,...,0,16,29,37,24,36,22,167,317,34.50


In [9]:
def calculate_ixg60(df):
    """
    Calculates individual expected goals per 60 minutes (ixG/60) for each player.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing player statistics with 'ixg' and 'toi' columns
        
    Returns:
        pd.DataFrame: Original DataFrame with new 'ixg60' column added
    """
    # Create copy of dataframe to avoid modifying original
    df = df.copy()
    
    # Calculate ixG/60 = (ixg / toi) * 60
    df['ixg60'] = (df['ixg'] / df['toi']) * 60
    
    return df

In [10]:
player_stats_2023_df = calculate_ixg60(player_stats_2023_df)
player_stats_2023_df


,player,team,position,gp,toi,goals,total_assists,first_assists,second_assists,total_points,...,penalties_drawn,giveaways,takeaways,hits,hits_taken,shots_blocked,faceoffs_won,faceoffs_lost,faceoffs_%,ixg60
0,Ryan Suter,DAL,D,82,1403.833333,2,14,6,8,16,...,0,28,14,75,92,86,0,0,-,0.159848
1,Jeff Carter,PIT,C,72,664.900000,5,3,1,2,8,...,3,16,16,75,32,25,169,87,66.02,0.528801
3,Brent Burns,CAR,D,82,1276.683333,5,12,5,7,17,...,5,43,30,34,33,72,0,0,-,0.223235
4,Corey Perry,"CHI, EDM",R,54,600.266667,9,7,4,3,16,...,14,17,15,42,66,17,0,1,0.00,0.756664
5,Joe Pavelski,DAL,C,82,1085.316667,12,25,14,11,37,...,19,51,19,70,45,61,299,281,51.55,0.804374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,Andrei Kuzmenko,"CGY, VAN",L,72,815.650000,14,13,9,4,27,...,15,21,24,12,68,16,1,0,100.00,0.807699
907,Connor Bedard,CHI,C,68,1043.150000,16,18,12,6,34,...,20,46,37,54,68,25,190,300,38.78,0.903609
908,Zach Benson,BUF,L,71,886.483333,10,16,10,6,26,...,13,18,30,35,90,30,4,10,28.57,0.733009
910,Leo Carlsson,ANA,C,55,788.383333,7,8,4,4,15,...,16,29,37,24,36,22,167,317,34.50,0.608841
